In [1]:
#Importing Packages
using Polymake #Via polymake.jl
using Test
using Base.Threads  # For parallel threads
using Polymake.polytope

polymake version 4.13
Copyright (c) 1997-2024
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
#Checking Polymake works properly
p = polytope.Polytope(POINTS=[1 -1 -1; 1 1 -1; 1 -1 1; 1 1 1; 1 0 0])
@show p.VERTICES
@show p.N_LATTICE_POINTS

p.VERTICES = pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1

p.N_LATTICE_POINTS = 9


9

In [3]:
# global variables initialization

const PMPolytope = typeof(polytope.cube(2))
MAX_LATTICE_POINTS = 50

# independent processing of each SmoothGeneration_output file
function process_polytopes(i)
    local_polytopes_list = PMPolytope[]
    try
        open("SmoothGeneration_output/SmoothGeneration_output$i") do f
            number_polytopes = 0
            polytopes_read = 0
            while !eof(f)
                number_vertices = parse(Int, readline(f))
                polytopes_read += 1

                vertices_matrix = zeros(Int, number_vertices, 4)
                for a in 1:number_vertices
                    vertices_string = readline(f)
                    vertex_vector = [1; parse.(Int64, split(vertices_string))]
                    vertices_matrix[a, :] .= vertex_vector
                end

                new_polytope = polytope.Polytope(POINTS=vertices_matrix)

                if !new_polytope.SMOOTH
                    for row in eachrow(new_polytope.VERTICES) 
                        println(join(row[2:end], " "))
                    end
                else
                    @assert new_polytope.NORMAL
                end

                if new_polytope.N_LATTICE_POINTS <= MAX_LATTICE_POINTS
                    isomorphic_found = false
                    for smooth_polytope in local_polytopes_list
                        if polytope.lattice_isomorphic_smooth_polytopes(smooth_polytope, new_polytope)
                            isomorphic_found = true
                            break
                        end
                    end
                    if !isomorphic_found
                        push!(local_polytopes_list, new_polytope)
                        number_polytopes += 1
                    end
                end
            end
            println(number_polytopes, " smooth polytopes extracted of ",
                    polytopes_read, " polytopes read from file SmoothGeneration_output$i")
        end
    catch e
        if isa(e, SystemError)
            println("Skipping file SmoothGeneration_output$i, does not exist/not found.")
        else
            rethrow(e)
        end
    end
    return local_polytopes_list
end

# Threaded run — store per-thread results
results_per_thread = [PMPolytope[] for _ in 1:nthreads()]

# Multi-threading
Threads.@threads for i in 4:2:28
    local_polys = process_polytopes(i)
    results_per_thread[threadid()] = vcat(get(results_per_thread, threadid(), PMPolytope[]), local_polys)
end

# Merge all the local lists into a unified output
polytopes_list = reduce(vcat, results_per_thread)

println("Total number of polytopes processed: ", length(polytopes_list))

4 smooth polytopes extracted of 4 polytopes read from file SmoothGeneration_output4
3108 smooth polytopes extracted of 16692 polytopes read from file SmoothGeneration_output6
2514 smooth polytopes extracted of 44305 polytopes read from file SmoothGeneration_output8
1471 smooth polytopes extracted of 18637 polytopes read from file SmoothGeneration_output10
2788 smooth polytopes extracted of 38764 polytopes read from file SmoothGeneration_output12
1527 smooth polytopes extracted of 29942 polytopes read from file SmoothGeneration_output14
983 smooth polytopes extracted of 17561 polytopes read from file SmoothGeneration_output16
150 smooth polytopes extracted of 2595 polytopes read from file SmoothGeneration_output18
37 smooth polytopes extracted of 443 polytopes read from file SmoothGeneration_output20
5 smooth polytopes extracted of 63 polytopes read from file SmoothGeneration_output22
2 smooth polytopes extracted of 57 polytopes read from file SmoothGeneration_output24
0 smooth polytope

In [4]:
#Outputting the Smooth 3-polytopes after creating the list
output_file = "prune_output" # Output Filename

if !isfile(output_file)
    touch(output_file)  # Create an empty file if it doesn't exist
else
    println("File exists! Possible error...")

end

for smooth_polytope in polytopes_list
    @assert smooth_polytope.SMOOTH
    @assert smooth_polytope.NORMAL
    
    open(output_file, "a") do file #Open the file in appending mode
        println(file, smooth_polytope.N_VERTICES)

        #Printing the contents of the matrix and avoiding the Polymake types showing up
        for row in eachrow(smooth_polytope.VERTICES) 
            println(file, join(row[2:end], " "))
        end
    end
end

In [5]:
# Use a dictionary to store counts keyed by number of interior lattice points
counts = Dict{Int, Int}()

for smooth_polytope in polytopes_list
    n = smooth_polytope.N_INTERIOR_LATTICE_POINTS
    if n >= 1
        counts[n] = get(counts, n, 0) + 1
    end
end

# Print total with at least one interior lattice point
total_with_interior = sum(values(counts))
println(total_with_interior, " with an interior lattice point")

# Print breakdown
for n in sort(collect(keys(counts)))
    word = n == 1 ? "point" : "points"
    println(counts[n], " with $n interior lattice $word")
end

println("Total polytopes: ", length(polytopes_list))

2070 with an interior lattice point
61 with 1 interior lattice point
227 with 2 interior lattice points
607 with 3 interior lattice points
564 with 4 interior lattice points
321 with 5 interior lattice points
221 with 6 interior lattice points
55 with 7 interior lattice points
13 with 8 interior lattice points
1 with 9 interior lattice points
Total polytopes: 12589


In [8]:
# Initialize a dictionary to store counts of lattice points
lattice_point_counts = Dict(i => 0 for i in 4:MAX_LATTICE_POINTS)

# Iterate through each polytope in the list
for P in polytopes_list
    num_lattice_points = P.N_LATTICE_POINTS
    @assert 4 <= num_lattice_points <= MAX_LATTICE_POINTS
    lattice_point_counts[num_lattice_points] += 1
end

# Now lattice_point_counts contains the number of polytopes with each number of lattice points
# from 4 to 44. To display the results, you can print the counts:

for num_points in 4:MAX_LATTICE_POINTS
    println("Lattice points = ", num_points, ": ", lattice_point_counts[num_points])
end

Lattice points = 4: 1
Lattice points = 5: 0
Lattice points = 6: 1
Lattice points = 7: 1
Lattice points = 8: 3
Lattice points = 9: 4
Lattice points = 10: 6
Lattice points = 11: 5
Lattice points = 12: 12
Lattice points = 13: 10
Lattice points = 14: 17
Lattice points = 15: 14
Lattice points = 16: 29
Lattice points = 17: 21
Lattice points = 18: 39
Lattice points = 19: 30
Lattice points = 20: 54
Lattice points = 21: 42
Lattice points = 22: 63
Lattice points = 23: 56
Lattice points = 24: 94
Lattice points = 25: 75
Lattice points = 26: 113
Lattice points = 27: 91
Lattice points = 28: 154
Lattice points = 29: 103
Lattice points = 30: 186
Lattice points = 31: 140
Lattice points = 32: 247
Lattice points = 33: 180
Lattice points = 34: 292
Lattice points = 35: 227
Lattice points = 36: 353
Lattice points = 37: 256
Lattice points = 38: 411
Lattice points = 39: 335
Lattice points = 40: 541
Lattice points = 41: 431
Lattice points = 42: 651
Lattice points = 43: 556
Lattice points = 44: 817
Lattice poin

In [9]:
p = polytopes_list[1200]
@show p.VERTICES
println(p.INTERIOR_LATTICE_POINTS)
p.VISUAL

p.VERTICES = pm::Matrix<pm::Rational>
1 0 0 0
1 0 1 0
1 8 0 0
1 3 1 0
1 0 0 1
1 28 0 1

pm::Matrix<pm::Integer>



LoadError: polymake: unknown property Polytope<Rational>::VISUAL at /home/mi/kylehuang/.julia/scratchspaces/d720cf60-89b5-51f5-aff5-213f193123e7/polymake_6703754860994480127_1.11_depstree_v2/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 432.
	Polymake::Core::BigObjectType::property(Polymake::Core::BigObjectType=ARRAY(0x43ba3858), "VISUAL") called at /home/mi/kylehuang/.julia/scratchspaces/d720cf60-89b5-51f5-aff5-213f193123e7/polymake_6703754860994480127_1.11_depstree_v2/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 714
	Polymake::Core::BigObjectType::encode_descending_path(Polymake::Core::BigObjectType=ARRAY(0x43ba3858), "VISUAL") called at /home/mi/kylehuang/.julia/scratchspaces/d720cf60-89b5-51f5-aff5-213f193123e7/polymake_6703754860994480127_1.11_depstree_v2/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 757
	Polymake::Core::BigObjectType::encode_read_request(Polymake::Core::BigObjectType=ARRAY(0x43ba3858), "VISUAL") called at /home/mi/kylehuang